# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
os.chdir("/Users/MX6367/Downloads")
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/Users/MX6367/Downloads
['/Users/MX6367/Downloads/event_data/2018-11-15-events.csv', '/Users/MX6367/Downloads/event_data/2018-11-22-events.csv', '/Users/MX6367/Downloads/event_data/2018-11-09-events.csv', '/Users/MX6367/Downloads/event_data/2018-11-18-events.csv', '/Users/MX6367/Downloads/event_data/2018-11-04-events.csv', '/Users/MX6367/Downloads/event_data/2018-11-01-events.csv', '/Users/MX6367/Downloads/event_data/2018-11-27-events.csv', '/Users/MX6367/Downloads/event_data/2018-11-10-events.csv', '/Users/MX6367/Downloads/event_data/2018-11-20-events.csv', '/Users/MX6367/Downloads/event_data/2018-11-17-events.csv', '/Users/MX6367/Downloads/event_data/2018-11-06-events.csv', '/Users/MX6367/Downloads/event_data/2018-11-03-events.csv', '/Users/MX6367/Downloads/event_data/2018-11-28-events.csv', '/Users/MX6367/Downloads/event_data/2018-11-12-events.csv', '/Users/MX6367/Downloads/event_data/2018-11-25-events.csv', '/Users/MX6367/Downloads/event_data/2018-11-26-events.csv', '/Users/MX6367/

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 
    
for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        for line in csvreader:
            full_data_rows_list.append(line) 

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("CREATE  KEYSPACE IF NOT EXISTS udacity \
    WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}")
except Exception as e:
    print("Failed to create keyspace:")
    print(e)


#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [9]:
## Query 1 Description:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
try:
    query = 'CREATE TABLE IF NOT EXISTS sparkify_sessions '
    query = query + '(session_id int, item_in_session int, artist text, song text, length float, PRIMARY KEY(session_id, item_in_session))'
    session.execute(query)
except Exception as e:
    print(e)
    

In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO sparkify_sessions (session_id, item_in_session, artist, song, length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
query = 'SELECT artist, song, length FROM sparkify_sessions WHERE session_id = 338 AND item_in_session = 4'
try:
    rows = session.execute(query).one()
    print(rows)
except Exception as e:
    print(e)
    


Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [13]:
# Query 2 Description: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
# for userid = 10, sessionid = 182
try:
    query = 'CREATE TABLE IF NOT EXISTS sparkify_userinfo '
    query = query + '(user_id int, session_id int, item_in_session int, song text, artist text, user_full_name text, PRIMARY KEY((user_id, session_id), item_in_session))'
    session.execute(query)
except Exception as e:
    print(e)
    
file = 'event_datafile_new.csv'
with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = 'INSERT INTO sparkify_userinfo (user_id, session_id, item_in_session, song, artist, user_full_name) '
        query = query + 'VALUES (%s, %s, %s, %s, %s, %s)'
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[9], line[0], line[1] + ' ' + line[4])) 
query = 'SELECT artist, song, user_full_name FROM sparkify_userinfo WHERE user_id = 10 AND session_id = 182'
try:
    row = session.execute(query).one()
    print(row)
except Exception as e:
    print(e)


Row(artist='Down To The Bone', song="Keep On Keepin' On", user_full_name='Sylvie Cruz')


In [20]:
# Query 3 Description: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
try:
    query = 'CREATE TABLE IF NOT EXISTS sparkify_song_popularity '
    query = query + '(song text, user_full_name text, user_id int, PRIMARY KEY(song, user_id))'
    # added user_full_name as a clustering column under song so that user_names would be sorted under a given song title
    # since we are interested in who listened to what song, not 
    session.execute(query)
except Exception as e:
    print(e)

file = 'event_datafile_new.csv'
with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = 'INSERT INTO sparkify_song_popularity (song, user_full_name, user_id) VALUES (%s, %s, %s)'
        session.execute(query, (line[9], line[1] + ' ' + line[4], int(line[10])))

query = "SELECT user_full_name FROM sparkify_song_popularity WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
    print(rows[0:2])
except Exception as e:
    print(e)

[Row(user_full_name='Jacqueline Lynch'), Row(user_full_name='Tegan Levine')]


### Drop the tables before closing out the sessions

In [21]:
try:
    session.execute('DROP TABLE sparkify_sessions')
except Exception as e:
    print(e)
    
try:
    session.execute('DROP TABLE sparkify_userinfo')
except Exception as e:
    print(e)
    
try:
    session.execute('DROP TABLE sparkify_song_popularity')
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()